<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - FunkSVD MRR</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, funksvd
from lenskit.algorithms.funksvd import FunkSVD
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(features,lrate,reg):
    all_recs = []
    test_data = []
    
    features=int(features)
    funksvd = FunkSVD(features=features, iterations=100, lrate=lrate, reg=reg, damping=5, range=None, bias=True) # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('FunkSVD', funksvd, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.recip_rank)
    results = rla.compute(all_recs, test_data)
                          
    return results["recip_rank"].mean()

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    features = params['features']
    lrate = params['lrate']
    reg = params['reg']
    metric = model_trainer(features,lrate,reg)
    print([features,lrate,reg,metric],',')
    return {'loss': -metric, 'status': STATUS_OK }

space={'features': hp.uniform('features', 1, 100),
       'lrate': hp.uniform('lrate', 0.0005, 0.005),
       'reg': hp.uniform('reg',0.005,0.05)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[80.5478498230622, 0.0049840535848296955, 0.038091839078643205, 0.0817612749451402]                                    
,                                                                                                                      
[88.27411838532592, 0.004142615944783315, 0.04581697393404878, 0.06621183111605364]                                    
,                                                                                                                      
[90.25760101818415, 0.0010012542305563987, 0.005943419309982733, 0.12342802531451742]                                  
,                                                                                                                      
[61.956680724560954, 0.002931587096858128, 0.036872969331247375, 0.1222313244036208]                                   
,                                                                                                                      
[14.40150183593618, 0.004861208457185009

[3.9221667543390133, 0.0015553488639189258, 0.010804873437817834, 0.2061377034229776]                                  
,                                                                                                                      
[19.883440866137605, 0.0011275869305045682, 0.008048260414663909, 0.2231198836233055]                                  
,                                                                                                                      
[98.5076225795257, 0.0008339169936320275, 0.012901327239962568, 0.050989934638137585]                                  
,                                                                                                                      
[13.978248600554398, 0.0020091086485697495, 0.009640582819178794, 0.17465251580853575]                                 
,                                                                                                                      
[39.64359013064693, 0.001418229116122517

In [4]:
array = [[80.5478498230622, 0.0049840535848296955, 0.038091839078643205, 0.0817612749451402]                                    
,                                                                                                                      
[88.27411838532592, 0.004142615944783315, 0.04581697393404878, 0.06621183111605364]                                    
,                                                                                                                      
[90.25760101818415, 0.0010012542305563987, 0.005943419309982733, 0.12342802531451742]                                  
,                                                                                                                      
[61.956680724560954, 0.002931587096858128, 0.036872969331247375, 0.1222313244036208]                                   
,                                                                                                                      
[14.40150183593618, 0.004861208457185009, 0.04494775572899891, 0.13214428758665772]                                    
,                                                                                                                      
[71.88507528362591, 0.004423209557217119, 0.023829925881030604, 0.10670359496480589]                                   
,                                                                                                                      
[6.989473185832646, 0.0009818778051998668, 0.044643591210397265, 0.16259954484336592]                                  
,                                                                                                                      
[36.541131621823645, 0.0005437642801388713, 0.028633765417929207, 0.06287025732970758]                                 
,                                                                                                                      
[17.084233402129865, 0.004023563260246417, 0.03894265523897744, 0.1253773069170986]                                    
,                                                                                                                      
[21.27190563012311, 0.0009055225778598514, 0.03394543981396956, 0.12289622605324635]                                   
,                                                                                                                      
[83.43039154481248, 0.0022197295837486954, 0.030160856766459687, 0.09766845826255574]                                  
,                                                                                                                      
[73.09238880082954, 0.0019647104936449684, 0.032578255858777314, 0.0994862211677047]                                   
,                                                                                                                      
[75.20031534555949, 0.004884281268268776, 0.01668300322212335, 0.10357236398349455]                                    
,                                                                                                                      
[51.50209903033901, 0.004977821553827462, 0.015034259219048222, 0.11474880247578582]                                   
,                                                                                                                      
[10.636186553479277, 0.0010768501895832886, 0.008164475272212791, 0.22199435644112694]                                 
,                                                                                                                      
[38.87911732868374, 0.0006785947922249593, 0.04863109672525043, 0.06188919792267526]                                   
,                                                                                                                      
[93.93954114261058, 0.004278435897445013, 0.037001699979482126, 0.07494077843477157]                                   
,                                                                                                                      
[8.620388136077471, 0.0038589139565926734, 0.030449881504528032, 0.16118260502248694]                                  
,                                                                                                                      
[9.121232656003091, 0.0012795444532631463, 0.036663564936180275, 0.1734404704962133]                                   
,                                                                                                                      
[1.1613888081335917, 0.0024155172499040548, 0.026030823922959623, 0.19637829427682268]                                 
,                                                                                                                      
[1.5049447871640327, 0.0031096910709303314, 0.005231677352834718, 0.19210936728294548]                                 
,                                                                                                                      
[28.80645550301749, 0.0017684656939888795, 0.010953162425326776, 0.21999705126560748]                                  
,                                                                                                                      
[28.768842383258402, 0.0016606438849758474, 0.010870381430085202, 0.22455206534462627]                                 
,                                                                                                                      
[27.422025898824405, 0.001422426027294731, 0.019605449638636918, 0.20234821221564694]                                  
,                                                                                                                      
[43.68729898587875, 0.0016309116430916065, 0.009365400660650813, 0.2166393683204178]                                   
,                                                                                                                      
[50.74463548985136, 0.0033912497327099225, 0.009751060009357894, 0.15869437845313022]                                  
,                                                                                                                      
[26.506756397800807, 0.0024884307827861664, 0.02053753170039568, 0.19240207897664235]                                  
,                                                                                                                      
[57.7279129323773, 0.0020564122717474068, 0.01219867181666524, 0.19084130342674965]                                    
,                                                                                                                      
[35.70734666840715, 0.0012076545391128974, 0.006730700699687887, 0.23442882018535963]                                  
,                                                                                                                      
[35.5057643053929, 0.001509768412897251, 0.015606065427583936, 0.20635264566087028]                                    
,                                                                                                                      
[45.589663001059314, 0.0005296997179447347, 0.005104249831786102, 0.07939668637115092]                                 
,                                                                                                                      
[64.8486254518733, 0.0025284295997279695, 0.01258031418159477, 0.17387097878347926]                                    
,                                                                                                                      
[23.892067939524104, 0.0018651011581131871, 0.020576331768847787, 0.20474184979351157]                                 
,                                                                                                                      
[31.697287061476853, 0.001252610683686976, 0.007025196215639868, 0.23620590954797396]                                  
,                                                                                                                      
[33.18618545484707, 0.001226913948062154, 0.023079865238725544, 0.1733469383006461]                                    
,                                                                                                                      
[56.02164762301163, 0.0007636061951573172, 0.017485050749676496, 0.08099297273616995]                                  
,                                                                                                                      
[18.239208319430453, 0.0036315248904167864, 0.0078024606705721904, 0.12838430464938694]                                
,                                                                                                                      
[43.74018739763572, 0.0028840452688661334, 0.014296472695867462, 0.17276251545081625]                                  
,                                                                                                                      
[39.02089969445465, 0.00215471250102408, 0.0070342059977700645, 0.21059536478638924]                                   
,                                                                                                                      
[63.71587046153805, 0.002694274741938226, 0.042223552127458056, 0.1031577193165509]                                    
,                                                                                                                      
[14.402147895226136, 0.0008304886947664227, 0.02571468297570673, 0.1350533051100398]                                   
,                                                                                                                      
[32.50966143872689, 0.0011499680831220814, 0.005349843091737969, 0.23188320345629404]                                  
,                                                                                                                      
[47.36756803352566, 0.0005148769925150181, 0.022635401381314566, 0.049497754525637844]                                 
,                                                                                                                      
[21.93314032142632, 0.0032509686440521917, 0.01830765909062057, 0.15917988521557297]                                   
,                                                                                                                      
[57.957984608196156, 0.0022914960393461093, 0.049938719285943844, 0.09431461758238831]                                 
,                                                                                                                      
[79.81732437267627, 0.001368527272290705, 0.013329425720534211, 0.1463426698544597]                                    
,                                                                                                                      
[68.09887815645907, 0.000913080050002056, 0.03994049344830013, 0.045231847730210724]                                   
,                                                                                                                      
[41.15067302235755, 0.004560657525724198, 0.0068110042415512, 0.12329648660938076]                                     
,                                                                                                                      
[52.62925213593163, 0.0006593980712675787, 0.03410488059064987, 0.049018676989263155]                                  
,                                                                                                                      
[17.974915332689953, 0.001040980721007818, 0.02787354393381816, 0.14132394746106805]                                   
,                                                                                                                      
[11.88961217186326, 0.0019221793820023451, 0.009036484247270798, 0.18637496517235155]                                  
,                                                                                                                      
[36.333704810446676, 0.0016149296236131808, 0.02967044742275385, 0.1602098232356521]                                   
,                                                                                                                      
[4.028591238633339, 0.002752143073806598, 0.016043833322684966, 0.18298807218264848]                                   
,                                                                                                                      
[87.72568854456175, 0.0030483910057999136, 0.025608728528672364, 0.10550967182460301]                                  
,                                                                                                                      
[47.018731811340125, 0.001817437023873537, 0.04673349478644588, 0.10761236570582847]                                   
,                                                                                                                      
[32.58264594307326, 0.002343773902371075, 0.03247256752703134, 0.17556398677347393]                                    
,                                                                                                                      
[6.371996349312351, 0.0046505779373367424, 0.010538188706124966, 0.15491780952010925]                                  
,                                                                                                                      
[76.7548901582085, 0.0020943264994045282, 0.021276376279926002, 0.13304743346810508]                                   
,                                                                                                                      
[70.43643226718068, 0.003898187463265092, 0.018740992596547427, 0.12234535819127604]                                   
,                                                                                                                      
[24.928156763414393, 0.0013169534075683227, 0.014831107392025198, 0.21624871180797997]                                 
,                                                                                                                      
[53.98519041057013, 0.0014843155552819206, 0.011775452654628852, 0.1951733413858381]                                   
,                                                                                                                      
[14.380930962459335, 0.0009778741173372928, 0.013915627404137435, 0.18647718917697756]                                 
,                                                                                                                      
[49.11524732217498, 0.000627344567333013, 0.017184181390580935, 0.06737601381198573]                                   
,                                                                                                                      
[30.638241629300378, 0.0035845178470471474, 0.006072976514894773, 0.15891842115243626]                                 
,                                                                                                                      
[60.86806968165158, 0.0026439427664090515, 0.0086436275285959, 0.18305657402961711]                                    
,                                                                                                                      
[21.64060096602288, 0.001066843975714961, 0.005138205859193003, 0.2377152267278704]                                    
,                                                                                                                      
[23.47065897051282, 0.0011601037310995039, 0.005054377814606673, 0.23944998477235827]                                  
,                                                                                                                      
[21.266919082973423, 0.0017256489588426032, 0.0051430674215626505, 0.22242595320868122]                                
,                                                                                                                      
[3.9221667543390133, 0.0015553488639189258, 0.010804873437817834, 0.2061377034229776]                                  
,                                                                                                                      
[19.883440866137605, 0.0011275869305045682, 0.008048260414663909, 0.2231198836233055]                                  
,                                                                                                                      
[98.5076225795257, 0.0008339169936320275, 0.012901327239962568, 0.050989934638137585]                                  
,                                                                                                                      
[13.978248600554398, 0.0020091086485697495, 0.009640582819178794, 0.17465251580853575]                                 
,                                                                                                                      
[39.64359013064693, 0.0014182291161225172, 0.011877084971796244, 0.20996929703239023]                                  
,                                                                                                                      
[24.36245295655965, 0.0007595028185433173, 0.006500463521551746, 0.1853014541495509]                                   
,                                                                                                                      
[28.92133314812559, 0.0022036163690433846, 0.02409608396760643, 0.19588709539502488]                                   
,                                                                                                                      
[8.19919826246868, 0.0005056939733407654, 0.007508186635782392, 0.14260634794954777]                                   
,                                                                                                                      
[43.882508292352675, 0.0012732132389979985, 0.015635515330949645, 0.18488668763442356]                                 
,                                                                                                                      
[11.279623273455789, 0.0016740014003116276, 0.019240150541858684, 0.18455493694737313]                                 
,                                                                                                                      
[17.40748044655104, 0.001104232555073084, 0.041500564546280275, 0.11710194222588163]                                   
,                                                                                                                      
[26.613490084246795, 0.000973701445513974, 0.005115044286857067, 0.22829381824309847]                                  
,                                                                                                                      
[34.57448629571587, 0.0006377551233519609, 0.009603657625652767, 0.11879656945726441]                                  
,                                                                                                                      
[41.16611712115888, 0.0025570425159630347, 0.014081302052215663, 0.1805450230018523]                                   
,                                                                                                                      
[30.316860859587198, 0.0017853110143061911, 0.01780869009917891, 0.20343092849739441]                                  
,                                                                                                                      
[21.863813910610123, 0.0019102137277262222, 0.021309204394617278, 0.1980559320753442]                                  
,                                                                                                                      
[3.187529355079434, 0.002827494248708197, 0.03586805159254826, 0.18060960522369662]                                    
,                                                                                                                      
[15.850928179599695, 0.0008583220805146725, 0.005854368663547879, 0.20959826964445008]                                 
,                                                                                                                      
[37.27071184750402, 0.004145844341677963, 0.011397446398457169, 0.13772699417099907]                                   
,                                                                                                                      
[10.605782925624643, 0.002338503827428213, 0.01044013659910192, 0.17697761269818169]                                   
,                                                                                                                      
[23.621561293772164, 0.0007413393388622704, 0.007884068310821299, 0.16971922421209207]                                 
,                                                                                                                      
[5.473209576224342, 0.0015021566164190752, 0.01655298352907552, 0.1963785583882854]                                    
,                                                                                                                      
[20.088971292416986, 0.0030135070056331293, 0.045225249037420256, 0.1636093706605066]                                  
,                                                                                                                      
[1.4449886850791138, 0.0032623969817336205, 0.012691610360118035, 0.18990569773468388]                                 
,                                                                                                                      
[30.899834785090807, 0.001218996080656538, 0.02661271305987496, 0.1569653386822866]                                    
,                                                                                                                      
[49.78274473761596, 0.0013792890055631078, 0.024239857107557158, 0.14618117493697796]                                  
,                                                                                                                      
[26.347494418495543, 0.0015867485277172941, 0.008700211829795034, 0.23214938384050482]                                 
,                                                                                                                      
[34.135945565453305, 0.0024562528517591805, 0.020185374163313157, 0.18925464501573142]                                 
,                                                                                                                      
[42.95124210293372, 0.0021289038803620295, 0.02905759398516348, 0.16788762655167652]                                   
,                                                                                                                      
[13.56075633055266, 0.0005580471086816413, 0.03221236119609671, 0.09614029470530833]                                   
,                                                                                                                      
[38.38625582379747, 0.002008832839022169, 0.022723766631236104, 0.18073994972015334]                                   
,                                                                                                                      
[18.774755298146168, 0.0010407620131531352, 0.014973657890752987, 0.1935076944146552] ]

In [5]:
metric = pd.DataFrame(array)
metric.columns = ['features','learning rate','reg','MRR']
metric.head(5)

,features,learning rate,reg,MRR
0,80.547850,0.004984,0.038092,0.081761
1,88.274118,0.004143,0.045817,0.066212
2,90.257601,0.001001,0.005943,0.123428
3,61.956681,0.002932,0.036873,0.122231
4,14.401502,0.004861,0.044948,0.132144


In [6]:
metric.sort_values(by=['MRR'],ascending=False).head(10)

,features,learning rate,reg,MRR
66,23.470659,0.001160,0.005054,0.239450
65,21.640601,0.001067,0.005138,0.237715
33,31.697287,0.001253,0.007025,0.236206
28,35.707347,0.001208,0.006731,0.234429
94,26.347494,0.001587,0.008700,0.232149
41,32.509661,0.001150,0.005350,0.231883
79,26.613490,0.000974,0.005115,0.228294
22,28.768842,0.001661,0.010870,0.224552
69,19.883441,0.001128,0.008048,0.223120
67,21.266919,0.001726,0.005143,0.222426


In [7]:
metric.to_csv('metric_MRR.csv', index=False)

In [2]:
metric = pd.read_csv('C:\\Users\\Alejo\\Tesis\\1M\\FunkSVD\\results\\metric_MRR.csv')
metric

,features,learning rate,reg,MRR
0,80.547850,0.004984,0.038092,0.081761
1,88.274118,0.004143,0.045817,0.066212
2,90.257601,0.001001,0.005943,0.123428
3,61.956681,0.002932,0.036873,0.122231
4,14.401502,0.004861,0.044948,0.132144
...,...,...,...,...
95,34.135946,0.002456,0.020185,0.189255
96,42.951242,0.002129,0.029058,0.167888
97,13.560756,0.000558,0.032212,0.096140
98,38.386256,0.002009,0.022724,0.180740


In [3]:
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
colormap = cm.hot
x = metric['reg']
y = metric['learning rate']
z = metric['features'] 
c = metric['MRR']


fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
img = ax.scatter(x, y, z, c=c, cmap=plt.hsv())
ax.set_xlabel('reg')
ax.set_ylabel('learning rate')
ax.set_zlabel('features')
fig.colorbar(img)
plt.show()